In [6]:
# Loading the Reuters dataset
from keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

In [7]:
# Encoding the data
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [8]:
# there is a built-in way to do this in Keras
from keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

In [9]:
import os
os.environ['PYTHONHASHSEED']=str(1)

import tensorflow as tf
import random
import numpy as np

def reset_random_seeds(SEED = 42):
  os.environ['PYTHONHASHSEED']=str(SEED)
  tf.random.set_seed(SEED)
  np.random.seed(SEED)
  random.seed(SEED)

In [12]:
from keras import models
from keras import layers

def create_and_train_model(hidden_layers=2,
                           hidden_units=64,
                           loss_function='categorical_crossentropy',
                           activation_function='relu'):
  
  model = models.Sequential()

  model.add(layers.Dense(hidden_units, activation=activation_function, input_shape=(10000,)))

  for i in range(hidden_layers - 1):
    model.add(layers.Dense(hidden_units, activation=activation_function))
 
  model.add(layers.Dense(46, activation='softmax'))

  model.compile(optimizer='rmsprop',
                loss=loss_function,
                metrics=['accuracy'])

  model.fit(x_train, one_hot_train_labels, epochs=8, batch_size=512)

  results = model.evaluate(x_test, one_hot_test_labels)

  return "[hidden_layers=%d, hidden_units=%d, loss_function=%s, " \
          "activation_function=%s] => loss: %.4f - accuracy: %.4f" % (
          hidden_layers, hidden_units, loss_function, activation_function,
          results[0], results[1])

In [ ]:
%%time

# Try using larger or smaller layers: 32 units, 128 units, and so on.

for value in [16, 32, 64, 128]:
  res = create_and_train_model(hidden_units=value)
  print(res)

Epoch 1/8
18/18 [==============================] - 1s 6ms/step - loss: 3.6341 - accuracy: 0.2622
Epoch 2/8
18/18 [==============================] - 0s 6ms/step - loss: 2.8315 - accuracy: 0.3715
Epoch 3/8
18/18 [==============================] - 0s 6ms/step - loss: 2.1837 - accuracy: 0.4244
Epoch 4/8
18/18 [==============================] - 0s 6ms/step - loss: 1.7659 - accuracy: 0.5994
Epoch 5/8
18/18 [==============================] - 0s 6ms/step - loss: 1.5142 - accuracy: 0.6910
Epoch 6/8
18/18 [==============================] - 0s 6ms/step - loss: 1.2997 - accuracy: 0.7233
Epoch 7/8
18/18 [==============================] - 0s 6ms/step - loss: 1.1743 - accuracy: 0.7425
Epoch 8/8
71/71 [==============================] - 0s 609us/step - loss: 1.2579 - accuracy: 0.7177
[hidden_layers=2, hidden_units=16, loss_function=categorical_crossentropy, activation_function=relu] => loss: 1.2579 - accuracy: 0.7177
Epoch 1/8
18/18 [==============================] - 1s 10ms/step - loss: 3.3338 - accur

In [ ]:
%%time

# You used two hidden layers. Now try using a single hidden layer, or three hidden layers.

for value in [1, 2, 3]:
  res = create_and_train_model(hidden_layers=value)
  print(res)